# Set up

In [2]:
import torch
import gpytorch
import pandas as pd
import numpy as np
import tqdm as tqdm


import os
import pickle

import GP_functions.Loss_function as Loss_function
import GP_functions.bound as bound
import GP_functions.Estimation as Estimation
import GP_functions.Training as Training
import GP_functions.Prediction as Prediction
import GP_functions.GP_models as GP_models
import GP_functions.Tools as Tools
import GP_functions.FeatureE as FeatureE

# Data

In [3]:
X_train = pd.read_csv('Data/X_train.csv', header=None, delimiter=',').values
X_test = pd.read_csv('LocalDisease/X_2.csv', header=None, delimiter=',').values

Y_train_pca = pd.read_csv('Data/Y_train_std_21.csv', header=None, delimiter=',').values
Y_test_21 = pd.read_csv('LocalDisease/Y_data_2_pca.csv', header=None, delimiter=',').values

train_x = torch.tensor(X_train, dtype=torch.float32)
test_x = torch.tensor(X_test, dtype=torch.float32)

train_y_21 = torch.tensor(Y_train_pca, dtype=torch.float32)
test_y_21 = torch.tensor(Y_test_21, dtype=torch.float32)



torch.set_default_dtype(torch.float32)

# Model

In [4]:
Device = 'cuda'

In [5]:
checkpoint = torch.load('multitask_gp_checkpoint_21.pth', map_location=Device)
model_params = checkpoint['model_params']

MVGP_models = GP_models.MultitaskVariationalGP(train_x, train_y_21, 
                                               num_latents=model_params['num_latents'], 
                                               num_inducing=model_params['num_inducing'], 
                                               covar_type=model_params['covar_type']).to(Device)

MVGP_models.load_state_dict(checkpoint['model_state_dict'])

MVGP_likelihoods = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=train_y_21.shape[1]).to(Device)
MVGP_likelihoods.load_state_dict(checkpoint['likelihood_state_dict'])

MVGP_models.eval()
MVGP_likelihoods.eval()

MultitaskGaussianLikelihood(
  (raw_task_noises_constraint): GreaterThan(1.000E-04)
  (raw_noise_constraint): GreaterThan(1.000E-04)
)

# PE

In [6]:
row_idx = 0

input_point = test_y_21[row_idx, :]

In [7]:
# local_train_x, local_train_y = Tools.find_k_nearest_neighbors_CPU(input_point, train_x, train_y_21, k=100)
local_train_x, local_train_y = Tools.find_k_nearest_neighbors_GPU(input_point, train_x, train_y_21, k=100)

In [8]:
preds_tmp = Prediction.preds_for_one_model(
    MVGP_models, MVGP_likelihoods, test_x[row_idx,:].unsqueeze(0).to(Device)
    ).cpu().detach().numpy()


In [9]:
preds_tmp

array([ 2.3394542e+00, -1.0232005e+01, -4.9106135e+00, -2.8437316e-01,
        2.4264023e+00, -1.0096259e+00,  4.6893984e-02,  3.2596797e-01,
       -1.3546339e-01,  1.0126385e-01, -2.5180405e-01, -5.3408200e-01,
        8.5156858e-03,  1.9775647e-01, -7.8717351e-02,  8.5906833e-02,
        1.9635315e-01, -1.6655783e-01,  2.0691767e-02, -2.7621374e-02,
        4.1130129e-02], dtype=float32)

In [10]:
input_point

tensor([ 2.2828e+00, -1.0191e+01, -4.9141e+00, -2.8206e-01,  2.3120e+00,
        -9.5108e-01,  2.4283e-02,  3.3189e-01, -9.8797e-02,  1.5036e-01,
        -2.4838e-01, -4.8423e-01,  4.2599e-02,  1.6839e-01, -7.5652e-02,
         6.0210e-02,  1.4430e-01, -1.1340e-01,  3.1942e-02, -7.4366e-03,
         5.6354e-02])

In [11]:
bounds = bound.get_bounds(local_train_x)

estimated_params_tmp, _ = Estimation.multi_start_estimation(
    MVGP_models, MVGP_likelihoods, row_idx, test_y_21, bounds,
    Estimation.estimate_params_for_one_model_Adam, num_starts=16, num_iterations=2000, lr=0.01,
    patience=10, attraction_threshold=0.1, repulsion_strength=0.1, device=Device
)

In [12]:
estimated_params_tmp

array([1.7292783 , 2.013312  , 0.96628624, 1.0797129 , 1.0090505 ,
       0.98289436, 0.9580273 , 1.0659508 , 1.0313327 , 0.97402185],
      dtype=float32)

In [13]:
test_x[row_idx, :]

tensor([1.7440, 2.0243, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000])